## ОСНОВНОЙ МОДУЛЬ
---
ОСНОВНОЙ МОДУЛЬ ПРЕДНАЗНАЧЕН СЧИТЫВАНИЯ ПОКАЗАНИЙ СО ВСЕХ СЕНСОРОВ ПРОЕКТА. ПОДГОТОВКИ ДАННЫХ ДЛЯ ОТОБРАЖЕНИЯ В ВИДЕ ГРАФИКОВ ЦИФРОВЫХ ПОКАЗАТЕЛЕЙ. ДЛЯ ФОРМИРОВАНИЯ 2D ГРАФИКОВ С СДАННЫМИ ПОСТУПАЮЩИМИ С МИКРОСХЕМЫ СЕНСОРА - ГИРОСКОПА, АКСЕЛЕРОМЕТРА, МАГНИТОМЕТРА

### Блок импорта библиотек необходимых для работы проекта

In [4]:
# Подключить библиотеки для мат обработки данных и генерации случайных данных
import numpy as np
import random
import math

# Подключить библиотеки для работы с 2D графиками
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator, FormatStrFormatter, AutoMinorLocator, NullFormatter

# Подключить библиотеки для работы анимацией
import matplotlib.animation as an

# Подключить библиотеки для работы с виджетами в Jupyter Notebook
from IPython.display import display
import ipywidgets as widgets

# Подключить библиотеки для работы с датой/временем
from datetime import datetime
import time
# Подключить библиотеки для асинхронной работы
import asyncio

# Подключить пользовательские модули
# ---
# Работа с интервалами в стиле setInterval JavaScript
from py.lib.lib_setinterval_rev01_v01 import setInterval
# Подключить модуль для работы с датчиком IMU ICM20948
from py.lib.lib_imu_20948_horizon_rev01_v03 import *
from py.lib.lib_figure2D_rev01_v01 import Figure2D

### Блок настроек среды исполнения Jupyter lab

In [3]:
# Задать режим работы отображения графической информации
%matplotlib  widget

# Инструкция требующая повторно перезагружать пользовательские функции после изменения их кода
%load_ext autoreload
%autoreload 2

### Блок визуализации 'сырых' данных Акселерометра

In [3]:
# Создать объект 'acc_options' содержащий настройки фигур Акселерометра
acc_options = {
                'figsize': (8, 8),
                'nrows': 3,
                'suptitle': 'ГРАФИК ПОКАЗАНИЙ АКСЕЛЕРОМЕТРА',
                'abscissa': 'N',
                'ordinate': ('X', 'Y', 'Z'),
                'facecolor': ('lightslategray', 'lightslategray', 'lightslategray'),
                'alpha': (0.5, 0.5, 0.5),
                'graphcolor': ('green', 'blue', 'red'),
                'xlim': (0, 50),
                'ylim': (-2, 2) }

In [ ]:
acc_grafics2D_raw = Figure2D( acc_options )
acc_grafics2D_raw.CreateVectorFigure()

grafics_acc_raw_anim = an.FuncAnimation(fig=acc_grafics2D_raw.Figure, func=acc_grafics2D_raw.Update, interval=200, frames=50) # frames=50, save_count=1500
# grafics_2D_anim = an.FuncAnimation(fig=grafics_2D.Figure, func=grafics_2D.UpdateGraph, interval=200, frames=50, save_count=1500) # сохраняет 1 500 кадров ?
plt.show()

#Codec = an.writers['ffmpeg']
#VideoParam = Codec(fps=15, bitrate=1800) #metadata={'artist': 'Me'}
#grafics_2D_anim.save('Line-Graph-Animation.mp4', VideoParam)



### Блок визуализации 'сырых' данных Гироскопа

In [ ]:
# Создать объект 'gyro_options' содержащий настройки фигур Гироскопа
gyro_options = {
                'figsize': (8, 8),
                'nrows': 3,
                'suptitle': 'ГРАФИК ПОКАЗАНИЙ ГИРОСКОПА',
                'abscissa': 'N',
                'ordinate': ('X', 'Y', 'Z'),
                'facecolor': ('lightslategray', 'lightslategray', 'lightslategray'),
                'alpha': (0.5, 0.5, 0.5),
                'graphcolor': ('green', 'blue', 'red'),
                'xlim': (0, 50),
                'ylim': (-2, 2) }

In [ ]:
gyro_grafics2D_raw = Figure2D( gyro_options )
gyro_grafics2D_raw.CreateVectorFigure()

grafics_gyro_raw_anim = an.FuncAnimation(fig=gyro_grafics2D_raw.Figure, func=gyro_grafics2D_raw.Update, interval=200, frames=50) # frames=50, save_count=1500
# grafics_2D_anim = an.FuncAnimation(fig=grafics_2D.Figure, func=grafics_2D.UpdateGraph, interval=200, frames=50, save_count=1500) # сохраняет 1 500 кадров ?
plt.show()

### Блок визуализации 'сырых' данных Магнитометра